In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [38]:
DF = pd.read_csv("../../datasets/lane_closure.csv",low_memory=False)
DF.head()

,Primary Street,Cross Street,Boundaries,Direction,Date Closed - From,Date Closed - To,Traffic Effect,Organization,Time Closed - From,Time Closed - To,...,Inserted Date,Geometry Id,Status,Complete Closure,KML,Latitude,Longitude,X,Y,Geometry
0,Bryce St,River Av,River Av to End,Eastbound & Westbound,August 16 2021,October 16 2021,"Eastbound curb lane, closed for building const...",Zelko,12:00 AM,12:00 AM,...,08/11/2021 06:37:43 PM,552963,Current,No,<LineString><extrude>0</extrude><tessellate>0<...,49.879699,-97.142781,633429.365245,5.526909e+06,MULTILINESTRING ((-97.142387643857 49.87915381...
1,Lilac St,Carter Av,Carter Av to Weatherdon Av,Southbound,June 14 2021,October 29 2021,"East side sidewalk, closed for building constr...",Globeland Construction Ltd,12:00 AM,12:00 AM,...,06/09/2021 10:35:09 AM,550863,Current,No,<LineString><extrude>0</extrude><tessellate>0<...,49.862454,-97.151445,632854.263781,5.524977e+06,MULTILINESTRING ((-97.151173092203 49.86211919...
2,Princess St,Pacific Av,Pacific Av to Rupert Av,Southbound,August 14 2020,October 16 2021,"West side sidewalk, covered by pedestrian scaf...",Hofer Const. Ltd.,12:00 AM,12:00 AM,...,05/21/2021 12:12:59 PM,550250,Current,No,<LineString><extrude>0</extrude><tessellate>0<...,49.901786,-97.139766,633584.982267,5.529370e+06,MULTILINESTRING ((-97.139915854251 49.90158994...
3,Henry Av,Austin St,Austin St to End,Westbound,May 10 2021,November 30 2021,"Westbound right turnaround lane, closed for st...",Bockstael Construction Limited,12:00 AM,11:00 PM,...,05/20/2021 09:32:11 AM,550156,Current,No,<LineString><extrude>0</extrude><tessellate>0<...,49.903340,-97.134152,633983.759759,5.529553e+06,MULTILINESTRING ((-97.13466284795 49.903503694...
4,Grandin St,Tache Av,Tache Av to St Joseph St,Eastbound,May 31 2021,January 13 2022,"South side sidewalk, closed for building const...",Qualico,12:00 AM,12:00 AM,...,05/26/2021 03:38:27 PM,550361,Current,No,<LineString><extrude>0</extrude><tessellate>0<...,49.896013,-97.126315,634566.885434,5.528752e+06,MULTILINESTRING ((-97.127616387474 49.89567885...


In [39]:
for col in DF.columns:
    print(col,DF[col].dtype)

Primary Street object
Cross Street object
Boundaries object
Direction object
Date Closed - From object
Date Closed - To object
Traffic Effect object
Organization object
Time Closed - From object
Time Closed - To object
Lane Closure ID int64
Modified Date object
Inserted Date object
Geometry Id int64
Status object
Complete Closure object
KML object
Latitude float64
Longitude float64
X float64
Y float64
Geometry object


In [40]:
DF = pd.read_csv("../../datasets/lane_closure.csv",low_memory=False)
DF = DF.rename(columns={"Primary Street":"Street","Cross Street":"At","Latitude":"Lat","Longitude":"Long"})
DF.loc[:,"Street"] = [" ".join(l[:-1]) for l in DF["Street"].str.split()]
DF.loc[:,"At"] = [" ".join(l[:-1]) for l in DF["At"].str.split()]
DF.loc[:,"Boundaries"] = DF["Boundaries"].str.split(" to ")
DF.loc[:,"Direction"] = DF["Direction"].str.split(" & ")
__clean_string = lambda l: [[[(float(j.split()[1]),float(j.split()[0])) for j in s.split(", ")] for s in m] for m in l.values]
DF.loc[:,"Geometry"] = __clean_string(DF["Geometry"].str.replace("MULTILINESTRING \(\(|\)\)","",regex=True).replace().str.split("\), \("))
DF["Date Closed - From"] = pd.to_datetime(DF["Date Closed - From"]) 
DF["Date Closed - To"] = pd.to_datetime(DF["Date Closed - To"]) 

def _num_points(m):
    set_i = set()
    for l in m: set_i |= set(l)
    return set_i 
DF["Num_Segments"] = [len(i) for i in DF["Geometry"].values]
DF["Num_Points"] = [len(_num_points(i)) for i in DF["Geometry"].values]

DF = DF.drop(columns=["Traffic Effect","Organization","Lane Closure ID","Modified Date","Inserted Date","Geometry Id","Status","KML","X","Y"])
DF

/Users/jasetran/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


,Street,At,Boundaries,Direction,Date Closed - From,Date Closed - To,Time Closed - From,Time Closed - To,Complete Closure,Lat,Long,Geometry,Num_Segments,Num_Points
0,Bryce,River,"[River Av, End]","[Eastbound, Westbound]",2021-08-16,2021-10-16,12:00 AM,12:00 AM,No,49.879699,-97.142781,"[[(49.879153816388, -97.142387643857), (49.880...",1,3
1,Lilac,Carter,"[Carter Av, Weatherdon Av]",[Southbound],2021-06-14,2021-10-29,12:00 AM,12:00 AM,No,49.862454,-97.151445,"[[(49.862119196838, -97.151173092203), (49.862...",1,5
2,Princess,Pacific,"[Pacific Av, Rupert Av]",[Southbound],2020-08-14,2021-10-16,12:00 AM,12:00 AM,No,49.901786,-97.139766,"[[(49.90158994935, -97.139915854251), (49.9019...",1,3
3,Henry,Austin,"[Austin St, End]",[Westbound],2021-05-10,2021-11-30,12:00 AM,11:00 PM,No,49.903340,-97.134152,"[[(49.903503694415, -97.13466284795), (49.9033...",1,4
4,Grandin,Tache,"[Tache Av, St Joseph St]",[Eastbound],2021-05-31,2022-01-13,12:00 AM,12:00 AM,No,49.896013,-97.126315,"[[(49.895678857718, -97.127616387474), (49.896...",1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,Sherbrook,Logan,"[Logan Av, William Av]",[Southbound],2021-09-20,2021-11-01,12:00 AM,12:00 AM,No,49.906584,-97.153705,"[[(49.904179261579, -97.155653416614), (49.904...",1,30
426,Gull Lake,Markham,"[Markham Rd, East Lake Dr]","[Northbound, Southbound]",2021-09-17,2021-10-08,12:00 AM,12:00 AM,No,49.809311,-97.158976,"[[(49.80730904568, -97.158914003228), (49.8081...",1,39
427,Wardlaw,Osborne,"[Osborne St, Scott St]",[Eastbound],2021-09-20,2021-10-15,12:00 AM,12:00 AM,No,49.877534,-97.142232,"[[(49.876961712509, -97.143938755153), (49.877...",1,5
428,Scotsborough,Beckinsale,"[Beckinsale By, Novavista Dr]",[Southbound],2021-09-23,2021-10-13,12:00 AM,12:00 AM,No,49.820905,-97.107388,"[[(49.820524519173, -97.107184511392), (49.820...",1,4


In [43]:
START_DATE = pd.to_datetime('Aug 1 2021'); END_DATE = pd.to_datetime('Sep 1 2021')
cond = (START_DATE > DF["Date Closed - From"]) & (DF["Date Closed - To"] >= END_DATE)
DF_AUG = DF[cond]
DF_AUG

,Street,At,Boundaries,Direction,Date Closed - From,Date Closed - To,Time Closed - From,Time Closed - To,Complete Closure,Lat,Long,Geometry,Num_Segments,Num_Points
1,Lilac,Carter,"[Carter Av, Weatherdon Av]",[Southbound],2021-06-14,2021-10-29,12:00 AM,12:00 AM,No,49.862454,-97.151445,"[[(49.862119196838, -97.151173092203), (49.862...",1,5
2,Princess,Pacific,"[Pacific Av, Rupert Av]",[Southbound],2020-08-14,2021-10-16,12:00 AM,12:00 AM,No,49.901786,-97.139766,"[[(49.90158994935, -97.139915854251), (49.9019...",1,3
3,Henry,Austin,"[Austin St, End]",[Westbound],2021-05-10,2021-11-30,12:00 AM,11:00 PM,No,49.903340,-97.134152,"[[(49.903503694415, -97.13466284795), (49.9033...",1,4
4,Grandin,Tache,"[Tache Av, St Joseph St]",[Eastbound],2021-05-31,2022-01-13,12:00 AM,12:00 AM,No,49.896013,-97.126315,"[[(49.895678857718, -97.127616387474), (49.896...",1,2
5,Edmonton,St Mary,"[St Mary Av, Graham Av]",[Northbound],2021-03-07,2021-10-22,11:00 PM,11:59 PM,No,49.890486,-97.146188,"[[(49.889751350198, -97.145788366233), (49.889...",1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,University,Dafoe,"[Dafoe Rd, Markham Rd]","[Northbound, Southbound]",2021-07-23,2021-10-29,12:00 AM,12:00 AM,Yes,49.808929,-97.142052,"[[(49.805997754019, -97.138668665141), (49.807...",1,28
370,Charlotte,,[At End],"[Eastbound, Westbound]",2021-06-28,2021-11-01,12:00 AM,12:00 AM,Yes,49.882473,-97.096175,"[[(49.882472182984, -97.096611651835), (49.882...",1,2
385,Keewatin,Logan,"[Logan Av, Selkirk Av]","[Northbound, Southbound]",2021-07-27,2021-10-22,12:00 AM,12:00 AM,No,49.926892,-97.195248,"[[(49.921824144979, -97.195915153849), (49.921...",3,61
397,Munroe,Henderson,"[Henderson Hw, Golspie St]",[Westbound],2021-05-03,2021-10-08,12:00 AM,12:00 AM,Yes,49.919450,-97.103062,"[[(49.92159361799, -97.109515392943), (49.9214...",1,7


In [44]:
# DF.to_csv("clean_datasets/LANE_CLOSURE.csv")
DF_AUG.to_csv("clean_datasets/LANE_CLOSURE_AUG_2021.csv")